In [294]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import stats
from shapely.geometry import Point
import geopandas
from pysal.lib import weights
import pygris.utils
import esda
from splot.esda import moran_scatterplot, lisa_cluster, plot_local_autocorrelation
from esda.moran import Moran_BV, Moran_Local_BV
from splot.esda import plot_moran_bv_simulation, plot_moran_bv
import PyQt5


In [295]:
local_path=r'C:/Users/ddrye/OneDrive/Documents/OMSA_Program/OMSA 2023\Summer2023/Practicum/off_git/data/'
df=pd.read_csv(local_path+r"2014_2020_all_tract.csv")
display(df.columns)
df['FIPS']=df['FIPS'].astype('str')
df=df.query('socioeconomic >= 0 & household_comp >= 0 & minority_status >= 0 & housing_type >= 0 and overall_svi >=0')
df=df[df['year']==2018]




Index(['longitude', 'latitude', 'FIPS', 'avg_co2', 'year', 'ST', 'STATE',
       'ST_ABBR', 'STCNTY', 'COUNTY', 'FIPS.1', 'LOCATION', 'year.1',
       'total_population', 'housing_units', 'num_households', 'unemployment',
       'socioeconomic', 'household_comp', 'minority_status', 'housing_type',
       'overall_svi', 'xco2_std', 'co2_1yr_change', 'co2_6yr_change',
       'co2_1yr_pct_change', 'co2_6yr_pct_change', 'co2_5yr_pct_change',
       'co2_4yr_pct_change', 'co2_3yr_pct_change', 'co2_2yr_pct_change'],
      dtype='object')

In [296]:
cens_tracts = geopandas.GeoDataFrame.from_file(r"C:\Users\ddrye\OneDrive\Documents\OMSA_Program\OMSA 2023\Summer2023\Practicum\off_git\cb_2020_us_tract_500k\cb_2020_us_tract_500k.shp")
cens_tracts['GEOID']=cens_tracts['GEOID'].str.lstrip('0')

In [297]:
df_merged  = cens_tracts.merge(df, left_on='GEOID', right_on='FIPS', how='right')
df_merged = shift_geometry(df_merged)

In [298]:
df_merged=df_merged[['GEOID', 'geometry','year','avg_co2','total_population','housing_units','num_households','unemployment','socioeconomic','household_comp','minority_status',
'housing_type','overall_svi','co2_1yr_change', 'latitude','longitude']]
#df_merged=df_merged.set_index('GEOID', drop=False)

In [299]:
df_merged['coords'] = list(zip(df_merged['longitude'],df_merged['latitude']))
df_merged['coords'] = df_merged['coords'].apply(Point)
df_merged = geopandas.GeoDataFrame(df_merged, geometry='coords', crs=df_merged.crs )
df_merged.drop(['latitude','longitude'], axis=1, inplace=True)
df_merged.dropna(inplace=True)

In [300]:
display(df_merged)

,GEOID,geometry,year,avg_co2,total_population,housing_units,num_households,unemployment,socioeconomic,household_comp,minority_status,housing_type,overall_svi,co2_1yr_change,coords
0,34005702808,"POLYGON ((1769199.204 480641.752, 1769441.368 ...",2018,408.250298,3383.0,1136.0,1058.0,164.0,0.5653,0.8186,0.7807,0.0617,0.4835,2.222560,POINT (-74.500 40.500)
1,39153504200,"POLYGON ((1202795.427 490329.128, 1202709.399 ...",2018,407.613775,1793.0,866.0,679.0,88.0,0.9667,0.7662,0.5766,0.4665,0.8189,7.649203,POINT (-81.500 41.500)
2,36071000502,"POLYGON ((1804960.654 657020.942, 1804895.560 ...",2018,407.924700,3570.0,1543.0,1093.0,81.0,0.8664,0.8827,0.9477,0.9450,0.9774,4.239578,POINT (-74.500 41.500)
3,6013370000,"POLYGON ((-2262654.076 364405.604, -2262618.28...",2018,409.004763,2814.0,1168.0,1091.0,66.0,0.2612,0.0870,0.8088,0.4539,0.3379,10.132632,POINT (-122.500 37.500)
5,5101180100,"POLYGON ((224839.627 -154157.125, 224780.042 -...",2018,407.839872,4040.0,2178.0,1376.0,52.0,0.5762,0.6644,0.0239,0.7992,0.5528,2.139056,POINT (-93.500 36.500)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68802,2090001200,"POLYGON ((-1930089.969 -906881.139, -1929990.2...",2018,407.039769,7869.0,3098.0,2941.0,244.0,0.2626,0.6814,0.2951,0.7400,0.4661,5.158241,POINT (-147.500 64.500)
69415,2170001100,"POLYGON ((-1956067.103 -1096330.111, -1955669....",2018,406.657928,7003.0,2010.0,1919.0,206.0,0.2316,0.3327,0.3842,0.3358,0.2517,2.434326,POINT (-149.500 61.500)
69855,2090001300,"POLYGON ((-1935609.715 -907807.110, -1935601.2...",2018,407.039769,6910.0,2488.0,2159.0,130.0,0.1282,0.0636,0.4168,0.6860,0.2300,5.158241,POINT (-147.500 64.500)
70250,2110000300,"POLYGON ((-1521268.707 -1197969.621, -1521283....",2018,406.897018,5925.0,2244.0,2177.0,238.0,0.2032,0.3116,0.5413,0.7720,0.4282,-5.102300,POINT (-134.500 58.500)


In [301]:
df_merged=df_merged.to_crs(epsg=3395)
w = weights.KNN.from_dataframe(df_merged, k=3)
w.transform = 'R'

C:\Users\ddrye\AppData\Roaming\Python\Python311\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 640 disconnected components.
  warnings.warn(message)


In [302]:
"""y=df_merged['avg_co2']
df_merged['lag']=weights.lag_spatial(w,y)
df_merged.head()"""

"y=df_merged['avg_co2']\ndf_merged['lag']=weights.lag_spatial(w,y)\ndf_merged.head()"

In [306]:
moran_loc_bv = Moran_Local_BV(df_merged['avg_co2'], df_merged['overall_svi'], w)

In [307]:
df_merged = geopandas.GeoDataFrame(df_merged, geometry='geometry', crs=df_merged.crs )
df_merged.plot()

<Axes: >

In [308]:
"""fig, ax = moran_scatterplot(moran_loc_bv, p=0.05)

lisa_cluster(moran_loc_bv, df_merged, p=0.05, figsize = (9,9))
plt.show()"""

plot_local_autocorrelation(moran_loc_bv, df_merged, 'overall_svi')
plot_local_autocorrelation(moran_loc_bv, df_merged, 'avg_co2')



(<Figure size 1500x400 with 3 Axes>,
 array([<Axes: title={'center': 'Moran BV Local Scatterplot'}, xlabel='Attribute', ylabel='Spatial Lag'>,
        <Axes: >, <Axes: >], dtype=object))